In [1]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pdb

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
train_pathN = "dataset/no_punctuation/1M/wiki_00"
train_pathP = "dataset/cured_text/1M/wiki_00"
valid_pathN = "dataset/no_punctuation/1M/wiki_01"
valid_pathP = "dataset/cured_text/1M/wiki_01"
test_pathN = "dataset/no_punctuation/1M/wiki_02"
test_pathP = "dataset/cured_text/1M/wiki_02"

In [18]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", " <eos>").split()

In [28]:
read_words(train_pathN)

['Andorra',
 '<eos>',
 'Andorra',
 'oficialmente',
 'Principado',
 'de',
 'Andorra',
 '(',
 ')',
 'es',
 'un',
 'pequeño',
 'país',
 'soberano',
 'del',
 'suroeste',
 'de',
 'Europa',
 'Constituido',
 'en',
 'Estado',
 'independiente',
 'de',
 'derecho',
 'democrático',
 'y',
 'social',
 'cuya',
 'forma',
 'de',
 'gobierno',
 'es',
 'el',
 'coprincipado',
 'parlamentario',
 'Su',
 'territorio',
 'está',
 'organizado',
 'en',
 'siete',
 'parroquias',
 'con',
 'una',
 'población',
 'total',
 'de',
 '<num>',
 '<num>',
 'habitantes',
 '(',
 '<num>',
 ')',
 'Su',
 'capital',
 'es',
 'Andorra',
 'la',
 'Vieja',
 '<eos>',
 'Tiene',
 '<num>',
 'km²',
 'de',
 'extensión',
 'territorial',
 'y',
 'está',
 'situado',
 'en',
 'los',
 'Pirineos',
 'entre',
 'España',
 'y',
 'Francia',
 'con',
 'una',
 'altitud',
 'media',
 'de',
 '<num>',
 'msnm',
 'Limita',
 'por',
 'el',
 'sur',
 'con',
 'España',
 '—',
 'con',
 'las',
 'comarcas',
 'catalanas',
 'de',
 'Cerdaña',
 'Alto',
 'Urgel',
 'y',
 'Pallar

In [19]:
def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)        
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))        

    words, _ = list(zip(*count_pairs))     
    word_to_id = dict(zip(words, range(len(words))))
    
    return word_to_id

In [20]:
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [29]:
def load_data():
    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_pathP)
    train_dataN = file_to_word_ids(train_pathN, word_to_id)
    train_dataP = file_to_word_ids(train_pathP, word_to_id)
    valid_dataN = file_to_word_ids(valid_pathN, word_to_id)
    valid_dataP = file_to_word_ids(valid_pathP, word_to_id)
    test_dataN = file_to_word_ids(test_pathN, word_to_id)
    test_dataP = file_to_word_ids(test_pathP, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    #print(train_data[:5])
    #print(word_to_id)
    #print(vocabulary)
    #print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary

train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary = load_data()

In [31]:
print(len(train_data))
print(" ".join([reversed_dictionary[x] for x in train_dataN[100:110]]))
print(" ".join([reversed_dictionary[x] for x in train_dataP[100:110]]))

161476
Sobirá — y por el norte con Francia — con
con España — con las comarcas catalanas de Cerdaña ,


In [54]:
class KerasBatchGenerator(object):

    def __init__(self, dataN, dataP, num_steps, batch_size, vocabulary, skip_step=5):
        self.dataN = dataN
        self.dataP = dataP
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset back to zero
        self.cnt1 = 0
        self.cnt2 = 0
        # skip_step is the number of words which will be skipped before the next batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.cnt1 + self.num_steps >= len(self.dataN):
                    # reset the index back to the start of the data set
                    self.cnt1 = 0
                    self.cnt2 = 0
                x[i, :] = self.dataN[self.cnt1 : self.cnt1 + self.num_steps]
                temp_y = self.dataP[self.cnt2 : self.cnt2 + self.num_steps]
                #print(" ".join([reversed_dictionary[x] for x in self.dataN[self.cnt1 : self.cnt1 + self.num_steps]]))
                #print(" ".join([reversed_dictionary[x] for x in self.dataP[self.cnt2 : self.cnt2 + self.num_steps]]))
                
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                                
                for i in range(self.cnt1, len(self.dataN)):
                    if reversed_dictionary[self.dataN[self.cnt1]] == "<eos>":
                        break
                    self.cnt1 += 1
                self.cnt1 += 1
                
                for i in range(self.cnt2, len(self.dataP)):
                    if reversed_dictionary[self.dataP[self.cnt2]] == "<eos>":
                        break
                    self.cnt2 += 1
                self.cnt2 += 1
            yield x, y

In [50]:
num_steps = 30
batch_size = 20

train_data_generator = KerasBatchGenerator(train_dataN, train_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_dataN, valid_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
hidden_size = 500
use_dropout=True

In [51]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 500)           10978000  
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 21956)         10999956  
_________________________________________________________________
activation_4 (Activation)    (None, 30, 21956)         0         
Total params: 25,981,956
Trainable params: 25,981,956
Non-trainable params: 0
________________________________________________________________

In [52]:
#1 to train, 2 to test
run_opt = 1
checkpointer = ModelCheckpoint(filepath="model-{epoch:02d}.hdf5", verbose=1)
num_epochs = 50

In [53]:
if run_opt == 1:
    model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])
    # model.fit_generator(train_data_generator.generate(), 2000, num_epochs,
    #                     validation_data=valid_data_generator.generate(),
    #                     validation_steps=10)
    model.save(data_path + "/final_model.hdf5")
    
elif run_opt == 2:
    model = load_model(data_path + "/model-40.hdf5")
    dummy_iters = 40
    example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Training data:")
    for i in range(dummy_iters):
        dummy = next(example_training_generator.generate())
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_training_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps-1, :])
        true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)
    # test data set
    dummy_iters = 40
    example_test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Test data:")
    for i in range(dummy_iters):
        dummy = next(example_test_generator.generate())
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_test_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps - 1, :])
        true_print_out += reversed_dictionary[test_data[num_steps + dummy_iters + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)

Epoch 1/50
Andorra <eos> Andorra oficialmente Principado de Andorra ( ) es un pequeño país soberano del suroeste de Europa Constituido en Estado independiente de derecho democrático y social cuya forma de
Andorra <eos> Andorra , oficialmente Principado de Andorra ( ) , es un pequeño país soberano del suroeste de Europa . Constituido en Estado independiente , de derecho , democrático
Andorra oficialmente Principado de Andorra ( ) es un pequeño país soberano del suroeste de Europa Constituido en Estado independiente de derecho democrático y social cuya forma de gobierno es
Andorra , oficialmente Principado de Andorra ( ) , es un pequeño país soberano del suroeste de Europa . Constituido en Estado independiente , de derecho , democrático y social
Tiene <num> km² de extensión territorial y está situado en los Pirineos entre España y Francia con una altitud media de <num> msnm Limita por el sur con España —
Tiene <num> km² de extensión territorial y está situado en los Pirineos , entre Espa

La actual normativa sobre revisiones médicas que se realizan a inmigrantes establece que se denegará la autorización de trabajo a aquellas personas que padezcan alcoholismo u otras toxicomanias enfermedades susceptibles
La actual normativa sobre revisiones médicas que se realizan a inmigrantes , establece que se denegará la autorización de trabajo a aquellas personas que padezcan alcoholismo u otras toxicomanias ,
Además se garantizan los Derechos Humanos con la figura del " Raonador del Ciutadà " ( defensor del Pueblo ) que es una institución que tiene como misión defender y
Además se garantizan los Derechos Humanos con la figura del " Raonador del Ciutadà " ( defensor del Pueblo ) , que es una institución que tiene como misión defender
Andorra se divide territorialmente en siete parroquias primer y único nivel político administrativo del país Aun así hay subdivisiones territoriales las parroquias de Ordino La Massana y San Julián de
Andorra se divide territorialmente en siete parroqu

El evento más importante en la vida cultural andorrana es el Festival internacional de jazz de Escaldes Engordany celebrado durante el mes de julio donde intérpretes como Miles Davis Fats
El evento más importante en la vida cultural andorrana es el Festival internacional de jazz de Escaldes - Engordany , celebrado durante el mes de julio , donde intérpretes como
La Orquesta Nacional de Cámara de Andorra dirigida y fundada en <num> por el violinista Gerard Claret celebra un certamen de canto con fama internacional avalado por Montserrat Caballé además
La Orquesta Nacional de Cámara de Andorra , dirigida y fundada en <num> por el violinista Gerard Claret , celebra un certamen de canto con fama internacional , avalado por
La música tradicional andorrana relacionada con la de otras regiones pirenaicas cuenta con danzas características como la " Marratxa " recopiladas por la folclorista Palmira Jaquetti i Isant <eos> En
La música tradicional andorrana , relacionada con la de otras regiones 

Su territorio reúne una gran diversidad de climas causada por una amplitud latitudinal que supera los <num> ° — incluyendo varias zonas geoastronómicas — una diferencia en la altitud que
Su territorio reúne una gran diversidad de climas , causada por una amplitud latitudinal que supera los <num> ° — incluyendo varias zonas geoastronómicas — , una diferencia en la
Su territorio continental americano que abarca gran parte del Cono Sur limita al norte con Bolivia y Paraguay al nordeste con Brasil al este con Uruguay y el océano Atlántico
Su territorio continental americano , que abarca gran parte del Cono Sur , limita al norte con Bolivia y Paraguay , al nordeste con Brasil , al este con Uruguay
Los primeros registros de pobladores en el actual territorio argentino se remontan al período Paleolítico La colonización española comenzó en <num> Argentina surgió como el estado sucesor del Virreinato del
Los primeros registros de pobladores en el actual territorio argentino se remontan al perío

El Principado de Andorra ( nombre local , " Principat d'Andorra " ) es un estado de la Europa meridional , situado en la cordillera de los Pirineos orientales .
Es un estado sin litoral en el que se encuentran importantes cruces de caminos de los Pirineos <eos> La superficie de Andorra englobada en la Península Ibérica es de <num>
Es un estado sin litoral , en el que se encuentran importantes cruces de caminos de los Pirineos . <eos> La superficie de Andorra , englobada en la Península Ibérica
La superficie de Andorra englobada en la Península Ibérica es de <num> km² Está en la vertiente meridional de los Pirineos Es una región de escarpados picos montañosos y estrechos
La superficie de Andorra , englobada en la Península Ibérica , es de <num> km² . Está en la vertiente meridional de los Pirineos . Es una región de escarpados
Por los valles de Andorra fluyen numerosos cursos de agua que se unen para formar el río Valira La cuenca del Valira tiene la forma de Y con sus dos
Por los valle

A los habitantes de Andorra que no disponen de la nacionalidad andorrana se les niega el derecho al voto y el derecho a presentarse a elecciones También tienen restringida su
A los habitantes de Andorra que no disponen de la nacionalidad andorrana se les niega el derecho al voto y el derecho a presentarse a elecciones . También tienen restringida
El mayor núcleo de población del Principado es la capital Andorra la Vieja con <num> habitantes ( <num> ) lo que representa el <num> de la población andorrana Las siguientes
El mayor núcleo de población del Principado es la capital , Andorra la Vieja , con <num> habitantes ( <num> ) , lo que representa el <num> de la población
El idioma oficial es el catalán lengua habitual del <num> de la población El castellano es la segunda lengua más hablada en el principado con un <num> de la población
El idioma oficial es el catalán , lengua habitual del <num> de la población . El castellano es la segunda lengua más hablada en el principado , con un <num

Emisoras de televisión <eos> <num> ( <num> ) Nota el gobierno promueve la emisión de canales tanto españoles como franceses dentro del territorio andorrano <eos> Aparatos de televisión <eos> <num>
Emisoras de televisión : <eos> <num> ( <num> ) Nota : el gobierno promueve la emisión de canales tanto españoles como franceses dentro del territorio andorrano . <eos> Aparatos de
<num> ( <num> ) Nota el gobierno promueve la emisión de canales tanto españoles como franceses dentro del territorio andorrano <eos> Aparatos de televisión <eos> <num> ( <num> ) <eos>
<num> ( <num> ) Nota : el gobierno promueve la emisión de canales tanto españoles como franceses dentro del territorio andorrano . <eos> Aparatos de televisión : <eos> <num> (
Aparatos de televisión <eos> <num> ( <num> ) <eos> Proveedores de servicios de Internet <eos> <num> ( <num> ) <eos> Usuarios de Internet <eos> <num> ( <num> ) <eos> Código
Aparatos de televisión : <eos> <num> ( <num> ) <eos> Proveedores de servicios de Internet :

  2/269 [..............................] - ETA: 36:56 - loss: 9.9935 - categorical_accuracy: 0.0800    En el Creciente Fértil del Oeste de Asia Egipto e India fueron los sitios de la primera siembra y cosecha hidráulica de plantas que habían sido recogidas previamente en la
En el Creciente Fértil del Oeste de Asia , Egipto e India fueron los sitios de la primera siembra y cosecha hidráulica , de plantas que habían sido recogidas previamente
En el año <num> a C la naciente agricultura llegó a Egipto Por lo menos desde <num> a C en el subcontinente indio se cultivó trigo y cebada como lo
En el año <num> a . C . , la naciente agricultura llegó a Egipto . Por lo menos desde <num> a . C . , en el subcontinente indio
En el año <num> a C la agricultura campesina se atrincheró en las orillas del Nilo Esto debido al poco desarrollo aún de las técnicas de riego Durante este tiempo
En el año <num> a . C . , la agricultura campesina se atrincheró en las orillas del Nilo . Esto debido al poco desar

  4/269 [..............................] - ETA: 32:25 - loss: 9.9928 - categorical_accuracy: 0.0579Las maquinarias son elementos que se utilizan para dirigir la acción realizada por las fuerzas de trabajo a base de energía por su parte en el campo agrícola los mecanismos
Las maquinarias son elementos que se utilizan para dirigir la acción realizada por las fuerzas de trabajo a base de energía ; por su parte en el campo agrícola ,
Los equipos agrícolas son un grupo de aparatos diseñados para abrir surcos en la tierra desmenuzar fumigar y fertilizar en el suelo <eos> Las herramientas agrícolas son instrumentos que se
Los equipos agrícolas son un grupo de aparatos diseñados para abrir surcos en la tierra , desmenuzar , fumigar y fertilizar en el suelo . <eos> Las herramientas agrícolas son
Las herramientas agrícolas son instrumentos que se utilizan para labrar la tierra cargar arena deshierbar remover la tierra abrir zanjas transportar abono o material etc Son muchas y muy variadas
Las he

KeyboardInterrupt: 